In [ ]:
import pandas as pd
google_df=pd.read_csv("/kaggle/input/enriched-global-mobility-data-apple-google/google_mobility.csv")


<h4> Here I try and find out the country regions how many are there</h4>

In [ ]:
len(google_df.country_region.unique())

In [ ]:
google_df["retail_and_recreation_percent_change_from_baseline"]

<h3> from this link <a href="https://www.google.com/covid19/mobility/data_documentation.html?hl=en#about-this-data">here</a>
we know that the baseline from which the columns retail,workplaces....The baseline is the median value, for the corresponding day of the week, during the 5-week period Jan 3–Feb 6, 2020.
<p>Let us assume the baseline to be 100 so -1 is basically 99 +1 is 101 ... and so on we change the columns</p>

In [ ]:
google_df["parks_percent_change_from_baseline"]

In [ ]:
def changeFromPercent(x):
    if x!=0:
        return (1+(x/100))*100

    else:
        return 100

In [ ]:
google_df["retail_and_recreation_percent_change_from_baseline"]=google_df["retail_and_recreation_percent_change_from_baseline"].apply(changeFromPercent)
google_df["grocery_and_pharmacy_percent_change_from_baseline"]=google_df["grocery_and_pharmacy_percent_change_from_baseline"].apply(changeFromPercent)
google_df["parks_percent_change_from_baseline"]=google_df["parks_percent_change_from_baseline"].apply(changeFromPercent)
google_df["transit_stations_percent_change_from_baseline"]=google_df["transit_stations_percent_change_from_baseline"].apply(changeFromPercent)
google_df["workplaces_percent_change_from_baseline"]=google_df["workplaces_percent_change_from_baseline"].apply(changeFromPercent)
google_df['residential_percent_change_from_baseline']=google_df['residential_percent_change_from_baseline'].apply(changeFromPercent)

In [ ]:
google_df

<h4> Just deleting some unwanted columns

In [ ]:
google_df.drop(["wikidata","sub_region_1","sub_region_2"],axis=1,inplace=True)

<h4> Now using datetime from python we seperate the datetime given into days months and year column for

In [ ]:
import datetime

string = "2020-02-15"
date = datetime.datetime.strptime(string, "%Y-%m-%d")
print(date.timetuple().tm_yday)
print(date.day)

<h4> knowing that from datetime I can easily get day,month,year I also extract the day of the year, so I can use it easily in plotting as a variable
<h4> so lets get to it!

In [ ]:
def stripthedate(x):
    return datetime.datetime.strptime(x, "%Y-%m-%d")
      
    
       

google_df["day"]=google_df["date"].apply(stripthedate)
google_df["month"]=google_df["date"].apply(stripthedate)
google_df["year"]=google_df["date"].apply(stripthedate)
google_df["day_in_year"]=google_df["date"].apply(stripthedate)

       

google_df["day"]=google_df["day"].apply(lambda x: int(x.day))
google_df["month"]=google_df["month"].apply(lambda x: int(x.month))
google_df["year"]=google_df["year"].apply(lambda x: int(x.year))
google_df["day_in_year"]=google_df["day_in_year"].apply(lambda x: int(x.timetuple().tm_yday))

In [ ]:
retail='retail_and_recreation_percent_change_from_baseline'

grocery='grocery_and_pharmacy_percent_change_from_baseline'
parks=   'parks_percent_change_from_baseline'
transit=  'transit_stations_percent_change_from_baseline'
workplace= 'workplaces_percent_change_from_baseline'
residential= 'residential_percent_change_from_baseline'

In [ ]:
india_df=google_df[google_df["country_region"]=="India"]
india_df["day"]

In [ ]:
#!pip install -U vega_datasets notebook vega

<h1> HERE WE NEED TO SET UP THE RENDER FOR ALTAIR
</h1>
<h4> I have made a utility script so you can add it  to your kernel as an utility script, <a href="https://www.kaggle.com/omegaji/altair-render-script"> click here</a>

In [ ]:

import altair_render_script

In [ ]:

import altair as alt
import json


#alt.renderers.enable('kaggle')
a=alt.Chart(india_df).transform_fold([grocery,parks,residential,workplace,transit],as_=["pro","values"]).mark_area().encode(
    alt.X('day_in_year:Q'),
    alt.Y('values:Q'),  alt.Color('pro:N'),
    alt.Row("pro:N"),
    ).properties(
    
    
).interactive(bind_y=False)
a


In [ ]:
alt.Chart(india_df.groupby(["month"]).sum().reset_index()).transform_fold([grocery,parks,residential,workplace,transit],as_=["pro","values"]).mark_bar().encode(
    alt.X('pro:O',axis=alt.Axis(labels=False),title=None),
    alt.Y('values:Q',axis=alt.Axis(labels=False)),  alt.Color('pro:N',title=None),
    alt.Column('month:N')
   ).interactive()

<h3>About the Apple Data
The CSV file and charts on this site show a relative volume of directions requests per country/region, sub-region or city compared to a baseline volume on 13 January 2020. We define our day as midnight-to-midnight, US Pacific time. Cities are defined as the greater metropolitan area and their geographic boundaries remain constant across the data set. In many countries/regions, sub-regions and cities, relative volume has increased since 13 January, consistent with normal, seasonal usage of Apple Maps. Day of week effects are important to normalise as you use this data. Data that is sent from users’ devices to the Maps service is associated with random, rotating identifiers so Apple does not have a profile of individual movements and searches. Apple Maps has no demographic information about users, so we cannot make any statements about the representativeness of our usage against the overall population.